In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import math
import matplotlib.pyplot as plt
import numpy as np

import PySpice.Logging.Logging as Logging
logger = Logging.setup_logging( logging_level='CRITICAL')

import os
import sys
from pathlib import Path
import IPython.display as ipd
from PySpice.Unit import *
from PySpice.Spice.Parser import SpiceParser
from PySpice.Spice.Netlist import Circuit, SubCircuit, SubCircuitFactory
from PySpice.Spice.Library import SpiceLibrary
from PySpice.Probe.Plot import plot
from PySpice.Doc.ExampleTools import find_libraries
from PySpice.Math import *
from PySpice.Plot.BodeDiagram import bode_diagram
from PySpice.Plot.BodeDiagram import bode_diagram_gain

import schemdraw
import schemdraw.elements as elm
from schemdraw import logic

directory_path = Path(os.path.abspath('')).resolve().parent.parent
spice_libraries_path = directory_path.joinpath("lib", "spice")
spice_library = SpiceLibrary(spice_libraries_path)
## set the project directory as directory_path
directory_path = Path(os.path.abspath('')).resolve()

In [ ]:
class VoltageDivider(SubCircuitFactory):
    __name__ = 'voltage_divider'
    __nodes__ = ('n1', 'n2', 'n3' )
    __R = 100@u_kΩ;
    def __init__(self, R=100@u_kΩ, w=0.4, name='voltage_divider'):
        self.__name__ = name
        self.__R = R
        super().__init__()
        self.R(1, 'n1', 'n2', R * w)
        self.R(2, 'n2', 'n3', R * (1.0-w) )
        
    def wiper(self, w) :
        if w == 0 :
            self.R1.resistance = self.__R * 0.0000001
            self.R2.resistance = self.__R * 0.9999999
        elif w == 1 :
            self.R1.resistance = self.__R * 0.9999999
            self.R2.resistance = self.__R * 0.0000001
        else :
            self.R1.resistance = self.__R * w
            self.R2.resistance = self.__R * (1.0-w)


# theory


In [ ]:
d = schemdraw.Drawing(unit=2.5, inches_per_unit=0.5, lw=1.1)

d += (op1 := elm.Opamp().at([5,0]).flip().label('$U_1$'))
d += (op2 := elm.Opamp().at([5,-5]).label('$U_2$'))
d += (op3 := elm.Opamp().at([5,-10]).label('$U_3$'))
d += (op4 := elm.Opamp().at([0,-5]).flip().label('$U_4$'))
d += (op5 := elm.Opamp().at([10,-8]).flip().label('$U_5$'))

#draw saw input
d += elm.Line(at=op1.in2).left().length(1)
d += elm.Dot()
d.push()
d += elm.Line().left().length(1).label('$v_{saw}$', loc='lft')
d.pop()

d += elm.Line().down().toy(op2.in2)
d.push()
d += elm.Line().right().tox(op2.in2)
d.pop()
d += elm.Dot()
d += elm.Line().down().toy(op3.in1)
d += elm.Line().right().tox(op3.in1)

#comperator voltages
d += elm.Line().left().at(op1.in1).length(7)
d += elm.Dot()
d.push()
d += elm.Resistor().down().toy(op4.in2).label('$R_1$\n100k')
d += elm.Dot()
d.push()
d += elm.Line().right().tox(op4.in2)
d.pop()
d += elm.Resistor().down().length(4).label('$R_2$\n100k')
d += elm.Line().toy(op3.in2)
d += elm.Dot()
d.push()
d += elm.Line().right().tox(op3.in2)
d.pop()
d += elm.Line().left().length(1).label('$CV_{position}$', loc='lft')
d.pop()
d += elm.Line().left().length(1).label('$CV_{width}$', loc='lft')

#connect U4 and U2
d += elm.Line().left().at(op4.in1).length(1)
d += elm.Line().down().length(1)
d.push()
d += elm.Resistor().right().length(4).label('$R_3$\n100k', loc='bottom')
d += elm.Line().up().toy(op4.out)
d += ( Dop4 := elm.Dot() )
d.pop()
d += elm.Dot()
d += elm.Resistor().down().label('$R_4$\n100k', loc='bottom')
d += elm.Ground()

d += elm.Line().at(op4.out).right().tox(Dop4.end)
d += elm.Line().up().toy(op2.in1)
d += elm.Line().right().tox(op2.in1)

#summ the second pulse
d += elm.Line().right().at(op2.out).length(1)
d += elm.Resistor().down().length(2.5).label('$R_5$\n100k')
d.push()
d += elm.Resistor().down().length(2.5).label('$R_6$\n100k')
d += elm.Line().left().length(1)
d.pop()
d += elm.Dot()
d += elm.Line().right().tox(op5.in2)

d += elm.Line().left().at(op5.in1).length(0.5)
d += elm.Ground()

#and the outputs
d += elm.Line().right().at(op1.out).length(1).label('$OUT_{1}$', loc='rgt')
d += elm.Line().right().at(op5.out).length(1).label('$OUT_{2}$', loc='rgt')

d.draw()




The CD4069UB device consist of six CMOS inverter circuits. These devices are intended for all generalpurpose inverter applications where the mediumpower TTL-drive and logic-level-conversion. This device is not ideal for linear amplification, but can be used as such and will add a lot of soft-clipping. Soft-clipping is expected from a valve. This makes this device special for musical usage and can be found in guitar distortion pedals [[2](http://www.runoffgroove.com/ubescreamer.html)] or the wasp filter [[3](https://www.schmitzbits.de/wasp.html)]. 
{: class="mt-6"}

In [ ]:
kicad_netlist_path = directory_path.joinpath('main', 'main.cir')
parser = SpiceParser(path=str(kicad_netlist_path))

results = []

#for i in np.arange(0, 5, 1) :
circuit = parser.build_circuit(ground=5)
circuit.include(spice_library['TL072c'])
circuit.include(spice_library['D1N4148'])
circuit.include(spice_library['LM13700/NS'])
circuit.include(spice_library['q2n3906'])

circuit.V('1', '+15V', circuit.gnd, 'DC 15')
circuit.V('2', '-15V', circuit.gnd, 'DC -15')
circuit.V('3', '/IN_1', circuit.gnd, 'DC 0 AC 0 Sine(0 5 400)')
circuit.V('4', '/CV_1', circuit.gnd, 'DC 0 AC 0 PULSE(0 5 0 20m 20m 1m 40m)')

#for c in ( VoltageDivider(R=100@u_kΩ, w=0.1, name='POT1'), 
#        VoltageDivider(R=500@u_kΩ, w=0.1, name='POT2') ) :
#    circuit.subcircuit(c)

simulator = circuit.simulator(temperature=25, nominal_temperature=25)
#simulator.options('INTERP')
#simulator.initial_condition("Net-_C5-Pad1_"=0.0)
analysis = simulator.transient(step_time=1@u_us, start_time=0@u_ms, end_time=40@u_ms)


fig_buffer, ax1_buffer = plt.subplots()

ax1_buffer.set_xlabel('time [ms]')
ax1_buffer.set_ylabel('amplitude [V]')
ax1_buffer.plot(u_ms(analysis['/CV_1'].abscissa), analysis['/CV_1'], color='Blue')
ax1_buffer.plot(u_ms(analysis['/OUT_1'].abscissa), analysis['/OUT_1'], color='Red')

ax1_buffer.legend(('CV [V]', 'Vout [V]'), loc=(0.75,0.8))

plt.tight_layout()
plt.show()

the chip should be powered by +5V/GND. With higher voltage the heat dissipation will be to big and the chip will be damaged. the chip is powered on pin 7 and 14 with +5V and Ground. the other pins can be used as amplifier. 

In [ ]:
import sympy as sp
V1, V2, R1, R2 = sp.symbols("V1 V2 R1 R2")
eq1 = sp.Eq(V2, (V1 * R1) / (R1 + R2))
eq1.rhs.evalf(subs={R1: 2.7@u_kOhm, R2: 22@u_kOhm, V1: 15@u_V})

{% include bom.html content="summe-main-bom" %}
{% include callouts.html %}
{% include reports.html%}

In [ ]:
from IPython.core.display import display, HTML
display(HTML('''
<div class="hero is-medium" style="background: url('{{ '/assets/cmos_buffer_files/tmb_ube_screamer.jpg' | relative_url }}') no-repeat center center; background-size: cover; background-attachment: fixed;">
   <div class="hero-body">
        <div class="content has-text-centered">
        </div>
    </div>
</div>
'''))

# references

